# 5300 Final: Orbits

Note: Some of the cells are a little slow to evaluate, sorry about that. You can run all at once and wait a bit and then just come back if you want, will work fine.

Here we solve for orbits due to the graviational force between two bodies. We will denote the position vectors of our two bodies of mass $m_1$ and $m_2$ as $\vec{r_1}$ and $\vec{r_1}$. The forces on $m_1$ and $m_2$ are respectively
    
$\vec{F_1}= \frac{Gm_1m_2}{|\vec{r_1}-\vec{r_2}|^3}(\vec{r_2}-\vec{r_1})$

$\vec{F_2}= \frac{-Gm_1m_2}{|\vec{r_1}-\vec{r_2}|^3}(\vec{r_2}-\vec{r_1})$


We note that this is a central force, so angular momentum is conserved. We will also consider orbits in the CM frame, or equivalently where the total momentum is zero. This means that the bodies will orbit each other in a plane (initial velocites will define some plane, to break out of this plane would require a change in angular momentum). Thus, we can reduce the problem to two coordinates (x and y) for each body rather than 3.

Our differential equations componentwise will be:

$\frac{d^2x_1}{dt^2} = \frac{Gm_2}{|\vec{r_1}-\vec{r_2}|^3}(x_2-x_1)$

$\frac{d^2y_1}{dt^2} = \frac{Gm_2}{|\vec{r_1}-\vec{r_2}|^3}(y_2-y1)$

$\frac{d^2x_2}{dt^2} = \frac{-Gm_1}{|\vec{r_1}-\vec{r_2}|^3}(x_2-x_1)$

$\frac{d^2y_2}{dt^2} = \frac{-Gm_1}{|\vec{r_1}-\vec{r_2}|^3}(y_2-y_1)$

where

$|\vec{r_1}-\vec{r_2}| = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2}$

We begin by setting up an orbit class:

## Orbit Class

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
class GravitationalOrbit():
    """"
    Implements and solves the differential equations for two bodies orbiting under gravitational attraction.
    
    Parameters:
    m1: (float) mass of first body
    m2: (float) mass of second body
    G: (float) gravitation constant
    
    Methods:
    dz_dt: Returns RHS of differential equation
    
    """
    
    def __init__(self, m1=1., m2=1., G=1.):
        self.m1 = m1
        self.m2 = m2
        self.G = G
    
    def dz_dt(self, t, z):
        
        """
        Returns dz/dt, where z is 8-component vector.

        Parameters:
        t: float
        z: float
            z[0] = x_1    z[1] = x_dot_1
            z[2] = y_1    z[3] = y_dot_1
            z[4] = x_2    z[5] = x_dot_2
            z[6] = y_2    z[7] = y_dot_2

        """
        r = np.sqrt((z[4]-z[0])**2+(z[6]-z[2])**2)

        return [ z[1], self.G*self.m2*(z[4]-z[0])/r**3, \
                 z[3], self.G*self.m2*(z[6]-z[2])/r**3, \
                 z[5], -self.G*self.m1*(z[4]-z[0])/r**3, \
                 z[7], -self.G*self.m1*(z[6]-z[2])/r**3 ]

    def solve_ode(self, t_pts, z_0,    
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        solution = solve_ivp(self.dz_dt, (t_pts[0], t_pts[-1]), 
                             z_0, t_eval=t_pts, method='RK23',
                             atol=abserr, rtol=relerr)
        x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2 = solution.y

        return x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2
    
    

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None, linewidth='norm'):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    
    elif linewidth=='thick':
        line, = ax.plot(x, y, label=label, 
            color=color, linestyle=linestyle, lw=3.)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

Now that we have our class and helper functions set up, let's try plotting some orbits! We will choose initial conditions so the total momentum is zero and we have orbits that look nice.

In [ ]:
# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 10.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

m1 = 1.
m2 = 5.
G = 1.

# Instantiate an orbit
o1 = GravitationalOrbit(m1 = m1, m2 = m2, G=G)

In [ ]:
#Initial conditions (define the initial positions and initial velocity for m1; initial velocity and position for m2 will  
# be defined to make COM velocity 0 and to put origin at COM)

x_1_0_l =     [1., 1., 1., 0.]
y_1_0_l =     [1., 0., 0., 1.]
x_dot_1_0_l = [-1., 0., 1., 0.3]
y_dot_1_0_l = [1., 1., 1., -1.]

# start the plot!
fig = plt.figure(figsize=(10,20))
overall_title = rf'Gravitational Orbit plots:  ' + \
                rf'm1 = {m1:.2}, m2 = {m2:.2}, G = {G:.2}'
fig.suptitle(overall_title, va='baseline')

nrow = len(x_1_0_l)

#Solve and plot for each initial condition
for i in range(0, nrow):
    
    x_1_0 = x_1_0_l[i]
    y_1_0 = y_1_0_l[i]
    x_dot_1_0 = x_dot_1_0_l[i]
    y_dot_1_0 = y_dot_1_0_l[i]
    x_2_0 = -(m1/m2)*x_1_0
    y_2_0 = -(m1/m2)*y_1_0
    x_dot_2_0 = -(m1/m2)*x_dot_1_0
    y_dot_2_0 = -(m1/m2)*y_dot_1_0

    z = [x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0]
    x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2 = o1.solve_ode(t_pts, z)
    
    # plot! 
    ax_a = fig.add_subplot(nrow,1,i+1)                  

    plot_y_vs_x(x_1, y_1, 
                axis_labels=['x', 'y'], 
                color='blue',
                label=rf'$m_1$', 
                title=rf'$\vec{{r_1(0)}} = ({x_1_0:.2}, {y_1_0:.2}),  \vec{{r_2(0)}} = ({x_2_0:.2}, {y_2_0:.2}), $' + \
                rf'$ \frac{{d\vec{{r_1}}}}{{dt}}(0) = ({x_dot_1_0:.2}, {y_dot_1_0:.2})$' + \
                rf'$ \frac{{d\vec{{r_2}}}}{{dt}}(0) = ({x_dot_2_0:.2}, {y_dot_2_0:.2})$',
                ax=ax_a)       
    plot_y_vs_x(x_2, y_2, 
                color='red',
                label=rf'$m_2$', 
                title=None,
                ax=ax_a) 
    ax_a.set_aspect(1)

fig.tight_layout()


## Reduce to Previously Considered Orbits

In class, we considered orbits where we moved to the center of mass frame and also used the relative coordinate. If we consider making one of the masses very heavy, we note that the COM frame will be essentially the same as the rest frame of the heavy mass (the position of the heavy mass will essentially be the COM). Thus, we can move to the COM frame as before and we will also be in the rest frame of the heavy mass with the heavy mass at the origin.

Since in this case the heavy mass will be stationary, the position of the light mass will be the same as the separation vector. So our plots should be the same as the plots from our previously considered orbits!

Let's try this out and compare to what we did in class earlier this semester. We will copy over an old orbit class, solve a particular set of initial conditions both ways, and overlay them:

In [ ]:
class Orbit:
    """
    Potentials and associated differential equations for central force motion
    with the potential U(r) = k r^n.  Several algorithms for integration of 
    ordinary differential equations are now available. 
    """
    
    def __init__(self, ang_mom, n, k=1, mu=1):
        self.ang_mom = ang_mom
        self.n = n
        self.k = k
        self.mu = mu
    
    def U(self, r):
        """Potential energy of the form U = kr^n."""
        return self.k * r**self.n
    
    def Ucf(self, r):
        """Centrifugal potential energy"""
        return self.ang_mom**2 / (2. * self.mu * r**2)
    
    def Ueff(self, r):
        """Effective potential energy"""
        return self.U(r) + self.Ucf(r)
    
    def U_deriv(self, r):
        """dU/dr"""
        return self.n * self.k * r**(self.n - 1)
        
    def Ucf_deriv(self, r):
        """dU_cf/dr"""
        return -2. * self.ang_mom**2 / (2. * self.mu * r**3)
        
    def Ueff_deriv(self, r):
        """dU_eff/dr"""
        return self.U_deriv(r) + self.Ucf_deriv(r)
        
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dr/dt d^2r/dt^2 dphi/dt]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            3-component vector with y[0] = r(t), y[1] = dr/dt, y[2] = phi
            
        """
        return [ y[1], 
                -1./self.mu * self.Ueff_deriv(y[0]), 
                self.ang_mom / (self.mu * y[0]**2) ]
    
    
    def solve_ode(self, t_pts, r_0, r_dot_0, phi_0,
                  method='RK23',
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Use solve_ivp with the option of specifying the method.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [r_0, r_dot_0, phi_0]  
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, method=method, 
                             atol=abserr, rtol=relerr)
        r, r_dot, phi = solution.y
        return r, r_dot, phi
    
    def solve_ode_Euler(self, t_pts, r_0, r_dot_0, phi_0):
        """
        Solve the ODE given initial conditions with the Euler method.
        The accuracy is determined by the spacing of times in t_pts.
        """
        
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for r, rdot, phi with zeros
        num_t_pts = len(t_pts)    # length of the array t_pts
        r = np.zeros(num_t_pts)
        r_dot = np.zeros(num_t_pts)
        phi = np.zeros(num_t_pts)
        
        # initial conditions
        r[0] = r_0
        r_dot[0] = r_dot_0
        phi[0] = phi_0
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            y = [r[i], r_dot[i], phi[i]]
            r[i+1] = r[i] + self.dy_dt(t,y)[0] * delta_t
            r_dot[i+1] = r_dot[i] + self.dy_dt(t,y)[1] * delta_t 
            phi[i+1] = phi[i] + self.dy_dt(t,y)[2] * delta_t
        return r, r_dot, phi   
    
    
    def solve_ode_Leapfrog(self, t_pts, r_0, r_dot_0, phi_0):
        """
        Solve the ODE given initial conditions with the Leapfrog method.
        """
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for r, rdot, r_dot_half, phi with zeros
        num_t_pts = len(t_pts)
        r = np.zeros(num_t_pts)
        r_dot = np.zeros(num_t_pts)
        r_dot_half = np.zeros(num_t_pts)
        phi = np.zeros(num_t_pts)
        
        # initial conditions
        r[0] = r_0
        r_dot[0] = r_dot_0
        phi[0] = phi_0
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            y = [r[i], r_dot[i], phi[i]]
            r_dot_half[i] = r_dot[i] + self.dy_dt(t, y)[1] * delta_t/2.
            r[i+1] = r[i] + r_dot_half[i] * delta_t
            
            y = [r[i+1], r_dot[i], phi[i]]
            r_dot[i+1] = r_dot_half[i] + self.dy_dt(t, y)[1] * delta_t/2.
            
            phi[i+1] = phi[i] + self.dy_dt(t,y)[2] * delta_t
        return r, r_dot, phi   
        
    
    def energy(self, t_pts, r, r_dot):
        """Evaluate the energy as a function of time"""
        return (self.mu/2.) * r_dot**2 + self.Ueff(r)

In [ ]:
# Common plotting time 
t_start = 0.
t_end = 10.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

#Make m2 really heavy
m1 = 1.
m2 = 1000.
G = 1.

# Instantiate an orbit our way
o1 = GravitationalOrbit(m1 = m1, m2 = m2, G=G)

#Initial conditions. We will choose the intial y to be zero for ease of calculations.
x_1_0_l =     [1., 1.]
y_1_0_l =     [0., 0.]
x_dot_1_0_l = [0., -5.]
y_dot_1_0_l = [40., 30.]

# start the plot!
fig = plt.figure(figsize=(10,20))
overall_title = rf'Gravitational Orbit plots:  ' + \
                rf'm1 = {m1:.2}, m2 = {m2:.2}, G = {G:.2}'
fig.suptitle(overall_title, va='baseline')

nrow = len(x_1_0_l)

#Solve and plot for each initial condition
for i in range(0, nrow):

    x_1_0 = x_1_0_l[i]
    y_1_0 = y_1_0_l[i]
    x_dot_1_0 = x_dot_1_0_l[i]
    y_dot_1_0 = y_dot_1_0_l[i]
    x_2_0 = -(m1/m2)*x_1_0
    y_2_0 = -(m1/m2)*y_1_0
    x_dot_2_0 = -(m1/m2)*x_dot_1_0
    y_dot_2_0 = -(m1/m2)*y_dot_1_0

    z = [x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0]
    x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2 = o1.solve_ode(t_pts, z)
    
        
    #Calculate angular momentum and reduced mass and instantiate other orbit class. Note mu will essentially be m1
    # but we'll go ahead calculate it. 
    
    ang_mom = m1*(x_1_0*y_dot_1_0 - y_1_0*x_dot_1_0) + m2*(x_2_0*y_dot_2_0 - y_2_0*x_dot_2_0)
    
    mu = (m1*m2)/(m1+m2)
    old1 = Orbit(ang_mom, n=-1, k=-G*m1*m2 , mu=mu)

    r_0 = x_1_0
    r_dot_0 = x_dot_1_0
    phi_0 = 0
    r_pts, r_dot_pts, phi_pts = old1.solve_ode(t_pts, r_0, r_dot_0, phi_0)
    

    #Convert back to x and y coordinates
    old_x = []
    old_y = []
    for j in range(0, len(r_pts)):
        old_x.append(r_pts[j]*np.cos(phi_pts[j]))
        old_y.append(r_pts[j]*np.sin(phi_pts[j]))

    
    # plot! 
    ax_a = fig.add_subplot(nrow,1,i+1)                  

    plot_y_vs_x(x_1, y_1, 
                axis_labels=['x', 'y'], 
                color='blue',
                label=rf'$m_1$', 
                title=rf'$\vec{{r_1(0)}} = ({x_1_0:.2}, {y_1_0:.2}),  \vec{{r_2(0)}} = ({x_2_0:.2}, {y_2_0:.2}), $' + \
                rf'$ \frac{{d\vec{{r_1}}}}{{dt}}(0) = ({x_dot_1_0:.2}, {y_dot_1_0:.2})$' + \
                rf'$ \frac{{d\vec{{r_2}}}}{{dt}}(0) = ({x_dot_2_0:.2}, {y_dot_2_0:.2})$',
                ax=ax_a)       
    plot_y_vs_x(x_2, y_2, 
                color='red',
                label=rf'$m_2$',
                linewidth='thick',
                title=None,
                ax=ax_a) 
    plot_y_vs_x(old_x, old_y, 
                color='orange',
                label=rf'old', 
                title=None,
                ax=ax_a) 
    ax_a.set_aspect(1)

fig.tight_layout()


### The orbits overlap almost exactly, so our hypothesis is supported.

We can also note that these orbits have the typical elliptical shape we expected from gravitational orbits in our prior treatment, with the origin (the heavy mass) at one of the foci.

Note the red dot corresponds to the mass m2, which just sits at the origin in this case.